##### this notebooks has the sample code for the qna generation functions and calculations checks with optimized code.

In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
import os
curpath = os.getcwd()
os.chdir(curpath.split("core")[0])

In [68]:
import openai
import json

from prompts import SIMPLE_NUMERICAL_MCQ_PROMPT, SIMPLE_TEXTUAL_TEXT_MCQ_PROMPT, SIMPLE_CONCEPTUAL_TEXT_MCQ_PROMPT, CALCULATION_CHECK_PROMPT
from helper import function_definitions, get_results_from_wolfram_cloud
from core.features.utils import add_dicts, calculate_cost_gpt4_omni
from core.features.provider import creator, text_creator_defaults


from dotenv import load_dotenv
load_dotenv()

True

### Functions

In [69]:
functions = [function_definitions["get_results_from_wolfram_cloud"]] 

functions

[{'name': 'get_results_from_wolfram_cloud',
  'description': 'Use this function for problems solvable with Wolfram Language code.',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'the input wolfram language query for mathematical calculations in string. ONLY proper wolfram language queries are supported. Format for Query: "a = Solve[aCoeff1*aVar == Var1 - Offset1 && aCoeff2*aVar == Var1 + Offset2, {aVar, Var1}][[1, 1, 2]]".'}}},
  'required': ['query']}]

### Conceptual Text MCQ Generation function

In [8]:
### theory will be the context text for conceptual questions. context will be used for textual/assessment type questions.

def generate_conceptual_text_mcqs(theory=None, n=1, examples = None):

    conversation = [{"role": "system", "content": SIMPLE_CONCEPTUAL_TEXT_MCQ_PROMPT}]

    if theory is None:
        raise ValueError("No theory provided")

    user_prompt = f"""
    TOPIC_THEORY:
    //theory//

    {theory}

    //theory//
   
    N: {n}

    EXAMPLE:
    {examples if examples else "Not Provided"}

    QUESTIONS: 
    """

    user_message = {'role': 'user', "content": user_prompt}
    conversation.append(user_message)
    
    total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}
    response = creator(**text_creator_defaults,
                    messages = conversation,
                    )
    
    total_usage = add_dicts(total_usage, dict(response.usage))
    output = response.choices[0].message.content
    
    return {"output": output, "total_usage": total_usage}

In [9]:
theory = """
Thermodynamics

Definitions:
1. System: A part of the universe under study.
2. Surroundings: Everything other than the system.
3. Thermodynamic State: The description of a system in terms of properties like pressure, volume, temperature, etc.
4. State Functions: Properties that depend only on the state of the system, not the path. Examples: U (Internal energy), H (Enthalpy), S (Entropy), and G (Gibbs free energy).
5. Path Functions: Depend on the path taken during a change, like heat (q) and work (w).
6. First Law of Thermodynamics: ΔU = q + w, where ΔU is the change in internal energy, q is heat, and w is work.
7. Isothermal Process: Process in which temperature remains constant.
8. Adiabatic Process: No heat exchange with surroundings. 

"""

examples = {
        "question": "Which of the following statements is true regarding thermodynamics?",
        "choices": {"a": "The surroundings refer to the part of the universe under study.", "b": "State Functions depend on the path taken during a change.", "c": "In an adiabatic process, there is no heat exchange with the surroundings.", "d": "The First Law of Thermodynamics is represented by the equation ΔU = q - w."},
        "answer": {"c": "In an adiabatic process, there is no heat exchange with the surroundings"}
    }

output = generate_conceptual_text_mcqs(
                        theory = theory,
                        n = 3
                        # examples = examples,
                        )

print(output['output'])


    {
        "mcqs": [
            {
                "question": "Which of the following is a state function in thermodynamics?",
                "choices": {
                    "a": "Heat",
                    "b": "Work",
                    "c": "Enthalpy",
                    "d": "Heat exchange"
                },
                "answer": {"c": "Enthalpy"}
            },
            {
                "question": "According to the first law of thermodynamics, if 100 J of heat is added to a system and the system does 60 J of work, what is the change in internal energy of the system?",
                "choices": {
                    "a": "40 J",
                    "b": "160 J",
                    "c": "100 J",
                    "d": "60 J"
                },
                "answer": {"a": "40 J"}
            },
            {
                "question": "What type of thermodynamic process occurs when there is no heat exchange between a system and its surroundings?",
        

In [12]:
print(output['total_usage'])

print(calculate_cost_gpt4_omni(output['total_usage']))

{'total_tokens': 876, 'completion_tokens': 274, 'prompt_tokens': 602}
0.01424


### Textual Text MCQ Generation function

In [17]:

# context will be the context text for textual questions. context will be used for textual/assessment type questions.
def generate_textual_text_mcqs(context=None, n=1, examples = None):

    if theory is None:
        raise ValueError("context must be provided")
    

    conversation = [{"role": "system", "content": SIMPLE_TEXTUAL_TEXT_MCQ_PROMPT}]

    user_prompt = f"""
    CONTEXT_TEXT:
    //context//

    {context}

    //context//

    N: {n}

    EXAMPLES:
    {str(examples) if examples else "Not Provided"}

    QUESTIONS: 
    """

    user_message = {'role': 'user', "content": user_prompt}
    conversation.append(user_message)

    total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}
    response = creator(**text_creator_defaults,
                        messages = conversation,
                    )
    
    total_usage = add_dicts(total_usage, dict(response.usage))

    output = response.choices[0].message.content

    return {"output": output, "total_usage": total_usage}


In [18]:
context = """Many United States companies have, unfortunately, made the search for legal protection from import competition into a major line of work. Since 1980 the United States International Trade Commission (ITC) has received about 280 complaints alleging damage from imports that benefit foreign governments’ subsidies. Another 340 charge that foreign companies “dumped” their products in the United States at “less than fair value.” Even when no unfair practices are alleged, the simple claim that an industry has been injured by imports is sufficient grounds to seek relief.

Contrary to the general impression, this quest for import relief has hurt more companies than it has helped. As corporations begin to function globally, they develop an intricate web of marketing, production, and research relationships, The complexity of these relationships makes it unlikely that a system of import relief laws will meet the strategic needs of all the units under the same parent company.

Internationalization increases the danger that foreign companies will use import relief laws against the very companies the laws were designed to protect. Suppose a United States-owned company establishes an overseas plant to manufacture a product while its competitor makes the same product in the United States. If the competitor can prove injury from the imports—and that the United States company received a subsidy from a foreign government to build its plant abroad—the United States company’s products will be uncompetitive in the United States, since they would be subject to duties.

Perhaps the most brazen case occurred when the ITC investigated allegations that Canadian companies were injuring the United States salt industry by dumping rock salt, used to de-ice roads. The bizarre aspect of the complaint was that a foreign conglomerate with United States operations was crying for help against a United States company with foreign operations. The “United States” company claiming the injury was a subsidiary of a Dutch conglomerate. In contrast, the “Canadian” companies included a subsidiary of a Chicago firm that was the second-largest domestic producer of rock salt."""

examples = {
        "question": "Which of the following options is most similar in meaning to the word: “Intricate” mentioned in the article?",
        "choices": {"a": "Twisted", "b": "Straightforward", "c": "Simple", "d": "Advance"},
        "answer": {"a": "Twisted"}
    }

result = generate_textual_text_mcqs(context, n = 3, examples = examples)

print(result["output"])
print(result["total_usage"])


    {
        "mcqs": [
            {
                "question": "What is the primary concern raised by the passage regarding United States companies seeking import relief?",
                "choices": {
                    "a": "The process is too complicated and expensive.",
                    "b": "It has been largely ineffective in protecting domestic industries.",
                    "c": "It has benefited more companies than it has harmed.",
                    "d": "The laws are outdated and need to be revised."
                },
                "answer": {"b": "It has been largely ineffective in protecting domestic industries."}
            },
            {
                "question": "According to the passage, what is a potential consequence of internationalization for United States-owned companies?",
                "choices": {
                    "a": "It simplifies their marketing strategies.",
                    "b": "It can lead to them being targeted by the same im

In [19]:
print(result["total_usage"])
print(calculate_cost_gpt4_omni(result["total_usage"]))

{'total_tokens': 1251, 'completion_tokens': 385, 'prompt_tokens': 866}
0.020210000000000002


### Numerical MCQ Generation function

In [72]:

def generate_numerical_mcqs(theory=None, n=1, examples = None):

    if theory is None:
        raise ValueError("Theory must be provided")

    conversation = [{"role": "system", "content": SIMPLE_NUMERICAL_MCQ_PROMPT}]

    user_prompt = f"""
    TOPIC_THEORY:
    //theory//

    {theory}

    //theory//

    N: {n}

    EXAMPLES:
    {str(examples) if examples else "Not Provided"}

    QUESTIONS: 
    """

    user_message = {'role': 'user', "content": user_prompt}
    conversation.append(user_message)

    total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}
    response = creator(**text_creator_defaults,
                        messages = conversation,
                        )
    
    total_usage = add_dicts(total_usage, dict(response.usage))

    output = response.choices[0].message.content

    return {"output": output, "total_usage": total_usage}


In [73]:
theory = """
Work Done in Various Processes:
a. Isothermal Process (constant temperature):
W = P(V2 - V1) * ln(V2 / V1)
b. Adiabatic Process (no heat exchange):
W = (P2V2 - P1V1) / (1 - γ), where γ is the adiabatic index
c. Isobaric Process (constant pressure):
W = P(V2 - V1)"""

examples = {
        "question": """Consider three processes: isothermal, adiabatic, and isobaric. For a certain gas, the following data is given:

Initial volume, V1 = 2 m^3
Final volume, V2 = 4 m^3
Initial pressure, P1 = 2 atm
Final pressure, P2 = 3 atm
Adiabatic index, γ = 1.4
Calculate the work done during the adiabatic process.""",
        "choices": {"a": "-1.5 atm.m^3", "b": "-2.0 atm.m^3", "c": "-2.5 atm.m^3", "d": "-20.0 atm.m^3"},
        "answer": {"d": "-20.0 atm.m^3"}
    }

result = generate_numerical_mcqs(theory = theory, n = 3, examples = examples)

In [74]:
print(result["output"])
print(result['total_usage'])

json_ques = json.loads(result['output'])

{
    "mcqs": [
        {
            "question": "A gas undergoes an isothermal expansion at a constant temperature. The initial and final volumes are 1.5 m^3 and 3 m^3 respectively, and the constant pressure throughout the process is 1 atm. Calculate the work done during this process.",
            "choices": {"a": "1.04 atm.m^3", "b": "2.08 atm.m^3", "c": "3.12 atm.m^3", "d": "4.16 atm.m^3"},
            "answer": {"a": "1.04 atm.m^3"}
        },
        {
            "question": "During an adiabatic compression, the volume of a gas changes from 5 m^3 to 2 m^3. The initial pressure is 1 atm, the final pressure is 3 atm, and the adiabatic index is 1.5. Calculate the work done in this process.",
            "choices": {"a": "1.2 atm.m^3", "b": "2.4 atm.m^3", "c": "3.6 atm.m^3", "d": "4.8 atm.m^3"},
            "answer": {"c": "3.6 atm.m^3"}
        },
        {
            "question": "A gas is held at a constant pressure of 2 atm while its volume is increased from 2 m^3 to 4 m^3. Cal

In [76]:
c1 = calculate_cost_gpt4_omni(result['total_usage'])
print(calculate_cost_gpt4_omni(result['total_usage']))

0.0183


In [77]:
json_ques['mcqs']

[{'question': 'A gas undergoes an isothermal expansion at a constant temperature. The initial and final volumes are 1.5 m^3 and 3 m^3 respectively, and the constant pressure throughout the process is 1 atm. Calculate the work done during this process.',
  'choices': {'a': '1.04 atm.m^3',
   'b': '2.08 atm.m^3',
   'c': '3.12 atm.m^3',
   'd': '4.16 atm.m^3'},
  'answer': {'a': '1.04 atm.m^3'}},
 {'question': 'During an adiabatic compression, the volume of a gas changes from 5 m^3 to 2 m^3. The initial pressure is 1 atm, the final pressure is 3 atm, and the adiabatic index is 1.5. Calculate the work done in this process.',
  'choices': {'a': '1.2 atm.m^3',
   'b': '2.4 atm.m^3',
   'c': '3.6 atm.m^3',
   'd': '4.8 atm.m^3'},
  'answer': {'c': '3.6 atm.m^3'}},
 {'question': 'A gas is held at a constant pressure of 2 atm while its volume is increased from 2 m^3 to 4 m^3. Calculate the work done during this isobaric process.',
  'choices': {'a': '2 atm.m^3',
   'b': '4 atm.m^3',
   'c': '6

#### calculation check for numerical mcqs

In [78]:
# calculation check and improvements

def calculation_checking(input):

    conversation = [{"role": "system", "content": CALCULATION_CHECK_PROMPT}]
    input_prompt = f"""

    INPUT: 

    {input}

    OUTPUT: 

    """ 

    input_message = {'role': 'user', 'content': input_prompt}
    conversation.append(input_message)

    total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}
    response = creator(model = text_creator_defaults["model"],
                        messages = conversation,
                        tools = [{"type": "function", "function": function_definitions["get_results_from_wolfram_cloud"]}],
                        tool_choice = "auto",
                        temperature = text_creator_defaults["temperature"]
                    )

    total_usage = add_dicts(total_usage, dict(response.usage))
    response = response.model_dump()
    response_message = response["choices"][0]["message"]
    tool_calls = response_message['tool_calls']

    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_results_from_wolfram_cloud": get_results_from_wolfram_cloud,
        }  # only one function in this example, but you can have multiple
        del response_message["function_call"]
        conversation.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call["function"]["name"]
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call["function"]["arguments"])
            function_response = function_to_call(
                query=function_args.get("query"),
            )
            conversation.append(
                {
                    "tool_call_id": tool_call["id"],
                    "role": "tool",
                    "name": function_name,
                    "content": str(function_response),
                }
            )

    # print(conversation)
    response = creator(**text_creator_defaults,
                        messages = conversation,
    )

    total_usage = add_dicts(total_usage, dict(response.usage))
    response = response.model_dump()
    output = response["choices"][0]["message"]["content"]
    return {"output": output, "total_usage": total_usage}

correct_ques = []
total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}
for ques in json_ques['mcqs']:
    # ques = str(ques).replace("'", '"')
    result = calculation_checking(ques)
    total_usage = add_dicts(total_usage, result["total_usage"])
    correct_ques.append(result["output"])
    print(result["output"])
    

{
"question": "A gas undergoes an isothermal expansion at a constant temperature. The initial and final volumes are 1.5 m^3 and 3 m^3 respectively, and the constant pressure throughout the process is 1 atm. Calculate the work done during this process.",
"choices": {"a": "1.5 atm.m^3", "b": "2.08 atm.m^3", "c": "3.12 atm.m^3", "d": "4.16 atm.m^3"},
"answer": {"a": "1.5 atm.m^3"}
}

    {
        "question": "During an adiabatic compression, the volume of a gas changes from 5 m^3 to 2 m^3. The initial pressure is 1 atm, the final pressure is 3 atm, and the adiabatic index is 1.5. Calculate the work done in this process.",
        "choices": {"a": "1.2 atm.m^3", "b": "2.0 atm.m^3", "c": "3.6 atm.m^3", "d": "4.8 atm.m^3"},
        "answer": {"b": "2.0 atm.m^3"}
    }

    {
    "question": "A gas is held at a constant pressure of 2 atm while its volume is increased from 2 m^3 to 4 m^3. Calculate the work done during this isobaric process.",
    "choices": {"a": "2 atm.m^3", "b": "4 atm.m^3

In [80]:
print(total_usage)
c2 = calculate_cost_gpt4_omni(total_usage)
print(c2)
print(c1 + c2)


{'total_tokens': 3470, 'completion_tokens': 638, 'prompt_tokens': 2832}
0.04746
0.06576


$0.065 (INR 5.4) is the cost of generating and checking the calculation of three mcqs